In [1]:
import numpy as np
import matplotlib.pyplot as plt
import noise
import random

# Directions (Up, Down, Left, Right)
DIRECTIONS = [(0, -1), (0, 1), (-1, 0), (1, 0)]


# Function to check if a cell is within bounds and a valid path
def in_bounds_and_path(x, y, width, height, maze):
    return 0 <= x < width and 0 <= y < height and maze[x, y] == 0


# Step 1: Generate Perlin noise for random blobs
def generate_perlin_blobs(width, height, scale=5.0):
    blobs = np.zeros((width, height))
    for x in range(width):
        for y in range(height):
            noise_value = noise.pnoise2(x / scale, y / scale, octaves=2)
            if noise_value > 0.1:  # Threshold for blobs
                blobs[x, y] = 0  # Path (open space)
            else:
                blobs[x, y] = 1  # Wall (solid space)
    return blobs


# Step 2: Apply iterative DFS to connect the blobs into a maze-like structure
def connect_blobs_with_dfs(blobs):
    width, height = blobs.shape
    visited = np.copy(blobs)

    # Use a stack instead of recursion
    stack = []

    # Start DFS from a random position inside the blobs
    start_x, start_y = random.choice(
        [
            (x, y)
            for x in range(1, width, 2)
            for y in range(1, height, 2)
            if blobs[x, y] == 0
        ]
    )
    stack.append((start_x, start_y))

    while stack:
        x, y = stack.pop()
        visited[x, y] = 0  # Mark as path

        # Shuffle directions for random exploration
        random.shuffle(DIRECTIONS)
        for dx, dy in DIRECTIONS:
            nx, ny = x + 2 * dx, y + 2 * dy

            if in_bounds_and_path(nx, ny, width, height, visited):
                visited[x + dx, y + dy] = 0  # Carve the wall between cells
                stack.append((nx, ny))  # Add the new position to the stack

    return visited


# Step 3: Combine Perlin blobs and DFS-carved paths
def generate_final_maze(perlin_blobs, dfs_paths):
    return np.minimum(perlin_blobs, dfs_paths)


# Create a white central area (5% of the maze) in the middle
def create_central_area(maze, percentage=0.05):
    width, height = maze.shape
    area_size = int(
        np.sqrt(percentage * width * height)
    )  # Calculate size based on 5% of total area
    center_x, center_y = width // 2, height // 2  # Center coordinates
    half_size = area_size // 2

    # Carve out a white (0) square in the middle of the maze
    maze[
        center_x - half_size : center_x + half_size,
        center_y - half_size : center_y + half_size,
    ] = 0
    return maze, (center_x, center_y)


# Define the maze dimensions
width, height = 51, 51

# Step 1: Generate Perlin noise-based blobs
perlin_blobs = generate_perlin_blobs(width, height, scale=5.0)

# Step 2: Use DFS to connect the blobs into a maze
dfs_paths = connect_blobs_with_dfs(perlin_blobs)

# Step 3: Combine Perlin noise blobs and DFS paths to create the final maze
final_maze = generate_final_maze(perlin_blobs, dfs_paths)

# Create a white central area in the middle (5% of maze size)
final_maze, center = create_central_area(final_maze, percentage=0.05)

# Plot the Perlin noise blobs
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(perlin_blobs, cmap="gray", interpolation="nearest")
plt.title("Perlin Noise Blobs")
plt.axis("off")

# Plot the DFS paths
plt.subplot(1, 3, 2)
plt.imshow(dfs_paths, cmap="gray", interpolation="nearest")
plt.title("DFS-Carved Paths")
plt.axis("off")

# Plot the final connected maze
plt.subplot(1, 3, 3)
plt.imshow(final_maze, cmap="gray", interpolation="nearest")
plt.title("Final Connected Maze")
plt.axis("off")

# Show all plots
plt.tight_layout()
plt.show()

KeyboardInterrupt: 